In [44]:
from localtileserver import get_folium_tile_layer
from localtileserver import TileClient
from folium import Map
import folium
import pandas as pd
import numpy as np


In [48]:
# prepare location data
import os
import json


def get_ngva_filenames(dir_path: str, search_name: str = 'C_Position_Row') -> list:
    """
    recursively searches in all subdirs of dir_path for ngva files that contain the search_name
    """
    res = []
    for root, dirs, files in os.walk(dir_path):
        for name in files:
            if search_name in name:
                res.append(os.path.join(root, name))
    
    return res

def ngva_to_df(ngva_filename: str) -> pd.DataFrame:
    with open(ngva_filename, 'r') as f:
        data = json.load(f)
        
    lat_lon_data = [
        {
            'lat': item.get("ddsData", {}).get("A_currentPosition", {}).get("A_latitude"),
            'lon': item.get("ddsData", {}).get("A_currentPosition", {}).get("A_longitude")
        } 
        for item in data if item.get("ddsData", {}).get("A_currentPosition", {}).get("A_latitude") is not None
    ]
    
    df = pd.DataFrame(lat_lon_data)
    
    return df
    

def load_position_data(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    return df

        

In [ ]:
import rasterio

def print_metadata(tif_path):
    with rasterio.open(tif_path) as src:
        print("Metadata for:", tif_path)
        print("Width and height:", src.width, src.height)
        print("Number of bands:", src.count)
        print("Coordinate reference system:", src.crs)
        print("Transform:", src.transform)
        print("Bounds:", src.bounds)
        for i in range(1, src.count + 1):
            band = src.read(i)
            print(f"Statistics for Band {i}: min={band.min()}, max={band.max()}, mean={band.mean()}")

In [ ]:
def plot_path_from_csv(map, lat_lon_df: pd.DataFrame, color="blue", name="Trajectory"):
    assert {'lat', 'lon'} in set(lat_lon_df.columns), "lat_lon_df must contain column 'lat' and column 'lon'"
    feature_group = folium.FeatureGroup(name=name, show=True)
    for index, row in lat_lon_df.iterrows():
        temp_df = lat_lon_df.iloc[[index]]
        html = temp_df.to_html(classes="table table-striped table-hover table-condensed table-responsive", index=False)
        popup = folium.Popup(html)

        tooltip_str = name
        folium.CircleMarker(
            location=[row['lat'], row['lon']],  
            radius=0.5,
            color=color,
            fill_color=color,
            fill_opacity=0.5,
            popup=popup,
            tooltip=tooltip_str
        ).add_to(feature_group)

    feature_group.add_to(map)
    return map


def add_base_raster(got_tif, name="Base", rgb=False):
    # First, create a tile server from local raster file
    tile_client = TileClient(got_tif)
    if rgb:
        band=[1,2,3]
    else:
        band = [1]
    # Create folium tile layer from that server
    t = get_folium_tile_layer(tile_client, band=band,name=name)
    map = Map(location=tile_client.center(),tiles=None,zoom_start=13)
    feature_group = folium.FeatureGroup(name=name,overlay=True)
    feature_group.add_child(t)
    feature_group.add_to(map)
    map.add_child(folium.LatLngPopup())

    return map


def add_raster(got_tif, map, name="Raster", rgb=False):
    # First, create a tile server from local raster file
    tile_client = TileClient(got_tif)
    if rgb:
        band=[1,2,3]
    else:
        band = [1]
    # Create folium tile layer from that server
    s = get_folium_tile_layer(tile_client, band=band,name=name)

    feature_group = folium.FeatureGroup(name=name,overlay=True)
    feature_group.add_child(s)

    feature_group.add_to(map)
    
    return map

In [ ]:
m = add_base_raster(r'../data/Ramot_Menashe_1.0m_24bit_WSG84.tif',rgb=True)
# m = add_raster(r"..\data\Ein-Quinia_elevation.tif", m)

data_files = get_ngva_filenames(r'/home/algo/Documents/1', 'C_Position_Row')

colors = ["red","green","blue","yellow","orange"]

for i, file in enumerate(data_files):
    data = ngva_to_df(file)
    data['lat'] = data['lat'].apply(lambda x : x * 180 / np.pi) - i * 0.001
    data['lon'] = data['lon'].apply(lambda x : x * 180 / np.pi) - i * 0.001
    m = plot_path_from_csv(m, data, color=colors[i], name=file)
    
folium.LayerControl().add_to(m)

m